In [1]:
import os

In [2]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [3]:
SEASONS = list(range(2016, 2023))

In [4]:
SEASONS

[2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [5]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [6]:
async def get_html(url, selector, sleep=5, retries=3):
    html=None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue 
        else:
            break
    return html
            

In [7]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [8]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com


In [9]:
standings_files = os.listdir(STANDINGS_DIR)

In [10]:
async def scrape_game(standings_file):
    with open(standings_file, "r") as f:
        html = f.read()

    soup = BeautifulSoup(html) 
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and "html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue 

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [11]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
        
    await scrape_game(filepath)

Lakers vs Clippers, March 8, 2020 | Basketball-Reference.com
Thunder vs Nets, January 7, 2020 | Basketball-Reference.com
Rockets vs Hawks, January 8, 2020 | Basketball-Reference.com
Celtics vs Knicks, October 20, 2018 | Basketball-Reference.com
Magic vs 76ers, October 20, 2018 | Basketball-Reference.com
Pistons vs Bulls, October 20, 2018 | Basketball-Reference.com
Hornets vs Heat, October 20, 2018 | Basketball-Reference.com
Timberwolves vs Mavericks, October 20, 2018 | Basketball-Reference.com
Suns vs Nuggets, October 20, 2018 | Basketball-Reference.com
Spurs vs Trail Blazers, October 20, 2018 | Basketball-Reference.com
Rockets vs Lakers, October 20, 2018 | Basketball-Reference.com
Hawks vs Cavaliers, October 21, 2018 | Basketball-Reference.com
Kings vs Thunder, October 21, 2018 | Basketball-Reference.com
Warriors vs Nuggets, October 21, 2018 | Basketball-Reference.com
Rockets vs Clippers, October 21, 2018 | Basketball-Reference.com
Magic vs Celtics, October 22, 2018 | Basketball-Refer

Error: net::ERR_INTERNET_DISCONNECTED at https://www.basketball-reference.com/boxscores/202012260DET.html
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/boxscores/202012260DET.html", waiting until "load"
============================================================